# Classification

This notebook introduces the Scikit-Learn interface for classification models.

## Setup

Load the packages and configure environment.

In [ ]:
import numpy as np
import pandas as pd

Using the `Default` dataset, which predicts the likelihood of a person defaulting on credit card payments based on their balance, income, and student status (yes / no).

In [ ]:
# download the data set directly from the web using pandas
url = "https://raw.githubusercontent.com/olearydj/INSY7120/refs/heads/main/notebooks/data/Default.csv"
default = pd.read_csv(url)

default.head()

## Simple Logistic Regression

First, we'll predict `default` based only on credit card `balance`, using logistic regression.

We'll use several components from SKL to achieve this.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

Get `X` and `y`. Note that the method below for `y` is exactly equivalent to using `get_dummies` as we've done before:

```python
y = pd.get_dummies(default['default'], drop_first=True, dtype=int)
```

For this simple binary case, the code below creates a boolean array based on the comparison `default == 'Yes'`. In Python `True` is equivalent to `1` and `False` to zero, and `astype(int)` converts them to that representation.

In [ ]:
# Extract feature and target
X = default[['balance']].values  # We need 2D array for sklearn
y = (default['default'] == 'Yes').astype(int)  # Convert to binary 0/1

# use value_counts to see class counts
y.value_counts()

There are 10,000 observations, 9667 of which are False (96.7%), and 333 are True (3.3%).

For this example, we'll use simple train-test split validation.

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

Logistic Regression uses gradient descent, which is sensitive to differences in scale between the features. Normalize them by z-score standardization using `StandardScaler`. Here we are taking care during fit to calculate the scaler parameters ($\mu$ and $\sigma$) using only the training data. The results are used to transform both the training and the test data. This avoids leaking information about the test data (e.g. scale) into the training data.

In [ ]:
# Standardize features
scaler = StandardScaler()

# fit the scaler using training data AND apply that transformation to the same
X_train_scaled = scaler.fit_transform(X_train)

# use the scaler coefficients from training to transform the test data
X_test_scaled = scaler.transform(X_test)

It is simple to fit the `LogisticRegression` model.

In [ ]:
# Train logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

Finally, make predictions. There are two methods used below. `model.predict` generates labels where `model.predict_proba` generates the probabilities. SKL's binary classifiers use a threshold of 0.5 for label assignment. Observations with a predicted probability of greater than 50% get the positive label.

In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]

Let's look at the relationship between probabilities, predictions, and the true labels.

In [ ]:
# Create a DataFrame to show samples with predictions and probabilities

# Get a sample of predictions
results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'probability': y_pred_prob
})

# Sort by probability to show examples around the threshold
results_df = results_df.sort_values('probability')

# Select informative examples (some below and some above the threshold)
threshold_examples = pd.concat([
    # Examples just below threshold (predict 0)
    results_df[(results_df['probability'] > 0.3) & (results_df['probability'] < 0.5)].head(3),
    # Examples just above threshold (predict 1)
    results_df[(results_df['probability'] >= 0.5) & (results_df['probability'] < 0.7)].head(3)
])

# Display with formatting
pd.set_option('display.precision', 4)
print("Prediction Examples (threshold = 0.5):")
print(threshold_examples)

You should be thinking, "what if I don't want to use a 50% threshold?" Good question and we will come back to it. TLDR; SKL doesn't support it directly, but it is an important consideration in model development.

Let's summarize the predictions:

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

You should be thinking, "what are all those numbers?" Good question, we'll come back to that. Included now for completeness.

### Interpreting the Predictions

These predictions are summarized above as a **Confusion Matrix**. In the case of binary classification it looks like this:

|            |                  |        **Predicted**         |        **Predicted**        |
| ---------- | ---------------- | :--------------------------: | :-------------------------: |
|            |                  |       **Negative (0)**       |      **Positive (1)**       |
| **Actual** | **Negative (0)** | True Negative (TN)<br>(2896) | False Positive (FP)<br>(10) |
| **Actual** | **Positive (1)** | False Negative (FN)<br>(70)  | True Positive (TP)<br>(24)  |

This table follows the scikit-learn format where:
- Rows represent actual values (0=No Default, 1=Default), the training labels
- Columns represent predicted values (0=No Default, 1=Default)

The four cells correspond to:

- **Top-Left (2896)**: True Negatives (TN) - Correctly predicted as "No Default"
- **Top-Right (10)**: False Positives (FP) - Incorrectly predicted as "Default"
- **Bottom-Left (70)**: False Negatives (FN) - Incorrectly predicted as "No Default"
- **Bottom-Right (24)**: True Positives (TP) - Correctly predicted as "Default"

This provides a complete picture of how the model's predictions align with the actual values, showing where the model succeeds and where it makes errors.

How would you interpret these results?

#### Answers - Hide Me!

The data has many more observations with a negative label (default = no) than positive. The first row of the CM shows 2896 + 10 = 2906 actual negatives and only 70 + 24 = 94 actual positives. That's a total of 3,000 observations, which matches the 30% test split we used.

If we think only in terms of the number of correct predictions, which are on the diagonal, this does quite well: (2896 + 24) / 3000 = 0.9733 → 97.3% correct.

It is also very good at identifying non-defaults (first row): 2896 / (2896 + 10) = 0.9966 → 99.7% correct.

Struggles to identify actual defaults (second row): 24 / (70 + 24) = 0.2553 → 25.5% correct.

Two types of errors, each with different frequencies and implications:

- False Positives (10) - predicting default when there is none
- False Negatives (70) - missing actual defaults

In these results, False Negatives are 7x more common than False Positives. In a lending business, failing to identify potential defaults (FNs) is typically more costly than incorrectly flagging bad borrowers (FPs). The high number of FNs suggests the model might not adequately address the business need despite seeming very accurate. Poor discriminator for this scenario.

Why might this model be biased towards predicting no default?

### Visualize the Results

Some fancy matplotlib work courtesy of Claude... Note that the actual and predicted dots should both appear on the 1.0 and 0.0 lines. They are shown stacked above and below those for clarity.

In [ ]:
# Visualize the results with better annotations
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.figure(figsize=(12, 7))

# Create small vertical offsets for clarity
offset_actual = 0.02  # Offset for actual values
offset_pred = -0.02   # Offset for predicted values

# Plot actual values (blue dots)
plt.scatter(X_test, y_test + offset_actual, color='blue', alpha=0.5, label='Actual')

# Plot predicted values (red dots)
plt.scatter(X_test, y_pred + offset_pred, color='red', alpha=0.5, label='Predicted')

# Plot the decision boundary
balance_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 100).reshape(-1, 1)
balance_range_scaled = scaler.transform(balance_range)
y_prob = model.predict_proba(balance_range_scaled)[:, 1]
plt.plot(balance_range, y_prob, 'g-', label='Probability of Default')
plt.axhline(y=0.5, color='k', linestyle='--', label='Decision Boundary')

# Create annotation boxes for False Negatives and False Positives
# False Negatives (top region)
fn_rect = patches.Rectangle((1000, 0.99), 930, 0.06, linewidth=2, edgecolor='r', facecolor='none')
plt.gca().add_patch(fn_rect)
plt.text(1500, 1.08, "False Negatives", color='red', fontsize=16, ha='center')

# False Positives (bottom right region)
fp_rect = patches.Rectangle((1955, -0.01), 600, 0.06, linewidth=2, edgecolor='r', facecolor='none')
plt.gca().add_patch(fp_rect)
plt.text(2250, -0.07, "False Positives", color='red', fontsize=16, ha='center')

# Adjust y-axis to accommodate offsets and annotations
plt.ylim(-0.15, 1.15)

plt.xlabel('Credit Card Balance')
plt.ylabel('Default Probability')
plt.title('Logistic Regression: Predicting Default based on Balance')
plt.legend()
plt.tight_layout()
plt.show()

We can observe a few things here:

- The decision boundary (dashed line) at 0.5 probability corresponds to a balance of approximately \$2000, where the model transitions from predicting "No Default" to "Default".
- Most accounts with balances below \$2k have low default probabilities and are classified as non-defaults (red dots at bottom)
- Most accounts with balances above \$2k have high default probabilities and are classified as defaults (red dots at top)
- There are several misclassifications evident:
  - False negatives (actual = default, prediction = non-default): blue dots at y=1 without matching red dot
  - False positives (actual = non-default, prediction = default): blue dots at y=0 without matching red dot

Most FNs occur with balances between ≈\$1k and 2k. FPs are less common and occur at higher balances.

### Interpret Model Coefficients

Extract model coefficients from the fitted model and interpret.

In [ ]:
# Print the model coefficients
print(f"Intercept: {model.intercept_[0]:.4f}")
print(f"Coefficient for balance: {model.coef_[0][0]:.4f}")

These don't match the results in ISL Chapter 4:

![](images/isl-tbl-4.1.png)

Why? We've standardized the features so they are in the scale of standard deviations of the feature values. Where ISL's result is interpreted in terms of $1 changes to the balance, our results must be interpreted as follows:

> For every one *standard deviation* increase in `balance`, the *log-odds* of `default` increase by 2.6678.

If we want to interpret the coefficients in the original scale, we need to reverse the effects of the `StandardScaler`. Here's the mathematical approach:

#### The Standardization Process

A feature $X$ is standardized using:

$$X_{scaled} = \frac{X - \mu_X}{\sigma_X}$$

Where $\mu_X$ is the mean and $\sigma_X$ is the standard deviation of $X$.

#### Logistic Regression with Standardized Data

For a model fit on standardized data:

$$\log\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 X_{scaled}$$

Substituting the standardization equation:

$$\log\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 \frac{X - \mu_X}{\sigma_X}$$

Rearranging:

$$\log\left(\frac{p}{1-p}\right) = \left[\beta_0 - \beta_1\frac{\mu_X}{\sigma_X}\right] + \frac{\beta_1}{\sigma_X}X$$

#### Coefficients in Original Scale

From this rearrangement, we can see:

1. The coefficient in original scale: $\beta_{1,orig} = \frac{\beta_1}{\sigma_X}$

2. The intercept in original scale: $\beta_{0,orig} = \beta_0 - \beta_1\frac{\mu_X}{\sigma_X}$

#### Interpretation

After transformation, we can properly interpret:
- For each 1-unit increase in the original feature, the log-odds increase by $\beta_{1,orig}$
- When the original feature equals zero, the log-odds equal $\beta_{0,orig}$

This manual transformation ensures we maintain the statistical benefits of standardization during model training while enabling interpretable coefficients in the original units.

In [ ]:
# Get the scaling parameters from the scaler
# first index, only one feature
std_dev_balance = scaler.scale_[0]
mean_balance = scaler.mean_[0]

# Convert the coefficient back to the original scale
coef_original = model.coef_[0][0] / std_dev_balance

# Convert the intercept back to the original scale
intercept_original = model.intercept_[0] - (model.coef_[0][0] * mean_balance / std_dev_balance)

print(f"Standard deviation of balance: {std_dev_balance:.4f}")
print(f"Mean of balance: {mean_balance:.4f}")
print(f"Coefficient in original scale: {coef_original:.4f}")
print(f"Intercept in original scale: {intercept_original:.4f}")
print(f"For every $1 increase in balance, log-odds of default increases by {coef_original:.4f}")
print(f"The log-odds of default when balance = 0 is {intercept_original:.4f}")

# The equation in the original scale would be:
print("\nLogistic regression equation in original scale:")
print(f"log-odds(default) = {intercept_original:.4f} + {coef_original:.4f} × balance")

These results are close to those obtained by ISL (-10.6513 + 0.0055 x balance). The difference in the intercept could come down to some combination of the splitting method, randomization, precision limits, and optimization method used.

## Multiple Logistic Regression

Also, introducing SKL pipelines.

Same problem, all predictors.

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify categorical and numerical columns
categorical_cols = ['student']
numerical_cols = ['balance', 'income']

SKL provides many workflow tools. The `Pipeline` class brings them all together, giving a way to represent all the steps in the modeling process concisely for repeatable, interpretable results.

The Pipeline encapsulates a complete machine learning workflow by sequentially applying a list of transformers and estimators. This provides several key benefits for engineers:

1. Code organization - keeps preprocessing, feature engineering, and model training steps in a single object
2. Prevention of data leakage - ensures transformations learned on training data are properly applied to test data
3. Parameter tuning - enables grid search across all pipeline components simultaneously
4. Production deployment - packages the entire workflow into a single estimator object with standard fit/predict methods

In this example we will chain preproccessing (scaling and dummy transformations) and classification together. This allows us to call fit() and predict() on the entire sequence as if it were a single estimator.

To achieve this we will first use `ColumnTransformer` to define the feature transformations for the numerical (`StandardScaler`) and categorical (`OneHotEncoder`) columns defined above. `OneHotEncoder` is the equivalent of `get_dummies` from pandas or the boolean mask approached we used early in this notebook. We use it here instead of those methods as `OneHotEncoder` is compatable with `Pipeline`.

The `ColumnTransformer` specifies a list of tuples, where each tuple includes the name, method, and target columns.

In [ ]:
# Create preprocessor for mixed data types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

Once the preprocessor is defined we can use it as a **step** in the `Pipeline`. The steps are simply a list of tuples, where each tuple includes the name and process to perform.

In [ ]:
# Create a pipeline with preprocessing and logistic regression
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

With the end-to-end process defined, preparing and splitting the data proceeds as normal.

In [ ]:
# Prepare X and y
X = default[numerical_cols + categorical_cols]
y = (default['default'] == 'Yes').astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

Rather than performing each step (scaling, fitting) separately, we fit the pipeline itself on the training data and make predictions using the test data. As before we use both `predict` to generate labels, and `predict_proba` for the associated probabilities.

In [ ]:
# Fit the model pipeline
model_pipeline.fit(X_train, y_train)

# Make predictions
y_pred = model_pipeline.predict(X_test)
y_pred_prob = model_pipeline.predict_proba(X_test)[:, 1]

Finally, we output the resulting confusion matrix and classification report scores.

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Note that there is only one change in this result - one prediction changes from FN to TP.

Now we extract and interpret the coefficients. The results of each step are stored in the `named_steps` attribute of the pipeline object. For example, the classifier's coefficients can be accessed via the usual `coef_` and `intercept_` attributes of the `classifier` step.

In [ ]:
# Get the coefficients from the model
coefficients = model_pipeline.named_steps['classifier'].coef_[0]

# Calculate the intercept in original scale
intercept = model_pipeline.named_steps['classifier'].intercept_[0]

# Get the column names after one-hot encoding
encoded_feature_names = numerical_cols + ['student_Yes']

# Create a DataFrame to display the coefficients
coef_df = pd.DataFrame({
    'Feature': encoded_feature_names,
    'Coefficient': coefficients
})

print("\nModel Coefficients:")
print(coef_df)

print(f"\nIntercept: {intercept:.4f}")

Interpreting these coefficients requires caution for several reasons:

1. Standardized coefficients: Since numerical predictors are standardized, the coefficients represent changes in log-odds per standard deviation increase, not per unit increase. This makes direct comparison between raw coefficients potentially misleading.
2. Log-odds scale: The coefficients represent changes in log-odds, which aren't intuitively interpretable like linear regression coefficients. Converting to odds ratios (by exponentiating) would make them more interpretable.
3. Feature correlation: Balance, income, and student status may be correlated, affecting the interpretation of individual coefficients.

The interpretation for each coefficient assumes that all other predictors are held constant.

With these caveats in mind, we can say:

- Balance appears to have the strongest effect, with each standard deviation increase associated with substantially higher default log-odds.
- Income has a much smaller positive coefficient, which may indicate that after controlling for balance, higher-income individuals might take on proportionally more risk.
- Student status shows a negative association with default, suggesting lower risk after accounting for balance and income.

Unlike our first model where we manually converted coefficients to the original scale, we haven't done this transformation for the pipeline model, making direct comparison with real-world values more difficult. For practical interpretation, we would need to transform these coefficients back to their original scales or consider calculating marginal effects at representative values.

Despite these interpretability challenges, the model's predictive performance remains the essential metric, and we observed only marginal improvement with the additional predictors.

### Addressing Confounding Effects

The coefficient for student is negative in the multiple logistic regression result. This is at odds with the single logistic regression result in ISL (pg 142, table 4.2), which shows a positive coefficient for the same (0.4049). That result does align with average balance and default rate by student status.

In [ ]:
# Let's examine the relationship between student status and balance
student_balance = default.groupby('student')['balance'].mean()
print("Average balance by student status:")
print(student_balance)

# Look at default rates by student status
default_by_student = default.groupby('student')['default'].apply(
    lambda x: (x == 'Yes').mean() * 100
)
print("\nDefault rate by student status:")
print(default_by_student.map(lambda x: f"{x:.2f}%"))

When we look at raw default rates without controlling for other factors, as in the single logistic regression model, students appear to have higher default rates overall. This would lead to the incorrect conclusion that being a student directly increases default risk. However, our multiple regression shows the opposite - being a student is associated with a *lower* probability of default when we control for balance and income.

This apparent contradiction occurs because of **confounding**, where a variable affects both independent and dependent variables, creating a misleading representation between them. That variable, called a confounder, can make the other two appear related when they are not, or hide a real relationship that exists.

1. Credit card balance is a confounding variable - it affects both student status (students tend to have higher balances) and default probability (higher balances increase default risk)
2. The negative coefficient reveals that after accounting for balance differences, students are actually *more responsible* borrowers than non-students in comparable financial situations
3. Without controlling for balance in our model, the relationship between student status and default risk would be obscured by the confounding effect

This example demonstrates why multiple regression is so valuable for causal inference - it helps us isolate the independent effect of each variable by controlling for confounding factors, revealing relationships that might be hidden or misrepresented in simpler analyses.

**Note:** We've seen this before - think back to single / linear regression where we were looking at the relationship between sales and advertising. We found that newspaper ad spend was correlated with radio advertising, leading so SLR predictions that radio advertising had a positive effect on sales. But in the MLR case we found the opposite effect. In that case, radio ads are a confounder because it has both a genuine effect on sales, but is also correlated with newspaper advertising. In the SLR case the apparent effect of newspaper ads is actually a side effect of radio ads. Again, MLR makes this clear by controlling for the other variables.


In [ ]:
# New section for 2D decision boundary
# Use balance and income as features
X_2d = default[['balance', 'income']].values
y = (default['default'] == 'Yes').astype(int)

# Split and scale as before
X_train_2d, X_test_2d, y_train_2d, y_test_2d = train_test_split(
    X_2d, y, test_size=0.3, random_state=42)

scaler_2d = StandardScaler()
X_train_2d_scaled = scaler_2d.fit_transform(X_train_2d)
X_test_2d_scaled = scaler_2d.transform(X_test_2d)

# Train model
model_2d = LogisticRegression()
model_2d.fit(X_train_2d_scaled, y_train_2d)

# Create a meshgrid to visualize decision boundary
def plot_decision_boundary(X, y, model, scaler):
    # Set up the mesh grid
    h = 0.01  # Mesh step size
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Scale the mesh
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    mesh_points_scaled = scaler.transform(mesh_points)
    
    # Predict on the mesh
    Z = model.predict(mesh_points_scaled)
    Z = Z.reshape(xx.shape)
    
    # Plot the decision boundary
    plt.figure(figsize=(12, 9))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
    
    # Plot the data points
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, 
                         edgecolors='k', cmap=plt.cm.coolwarm)
    
    # Add labels and legend
    plt.xlabel('Balance ($)')
    plt.ylabel('Income ($)')
    plt.title('Decision Boundary for Default Prediction')
    plt.colorbar(scatter, label='Default Status')
    
    # Add contour lines to emphasize boundary
    plt.contour(xx, yy, Z, [0.5], linewidths=2, colors='k')
    
    # Annotate regions
    x_center = (x_max + x_min) / 2
    y_text_pos = y_max - 0.2 * (y_max - y_min)
    # Find a point in each region for text
    i, j = np.unravel_index(Z.argmin(), Z.shape)
    plt.text(xx[i, j], yy[i, j] - 0.3, "Predict: No Default", 
             ha='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))
    
    i, j = np.unravel_index(Z.argmax(), Z.shape)
    plt.text(xx[i, j], yy[i, j] - 0.3, "Predict: Default", 
             ha='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    return plt

# Plot the decision boundary
plot_decision_boundary(X_2d, y, model_2d, scaler_2d)